# SAG workflow with Homomorphic Encryption

In this example, we will demonstrate how to use homomorphic encryption (HE)
by building on the previous [FedAvg with SAG workflow](../sag/sag.ipynb#title) example using the CIFAR10 dataset.

## Homomorphic Encryption

Homomorphic encryption is a form of encryption that allows computations to be performed on encrypted data, which is especially useful for preserving privacy in a federated learning setting.
FLARE uses the [TenSEAL](https://github.com/OpenMined/TenSEAL) library to enable homomorphic encryption operations on tensors in the provided FLARE [HE implementation](https://github.com/NVIDIA/NVFlare/tree/main/nvflare/app_opt/he).

## Job Configuration

On the client side, to configure homomorphic encryption we must add the following HE filters in config_fed_client.conf:

- `HEModelDecryptor`: Data filter to decrypt incoming Shareable objects using HE.
- `HEModelEncryptor`: Result filter to encrypt outgoing Shareable objects using HE.

On the server side, we configure components with HE support in config_fed_server.conf:

- `HEModelShareableGenerator`: This ShareableGenerator converts between Shareable and Learnable objects using HE.
- `HEInTimeAccumulateWeightedAggregator`: Filter to encrypt Shareable object using HE.
- `HEModelSerializeFilter`: Used to serialize TenSEAL encrypted server models for use with HE support.

Let's use the Job CLI to create the job from the sag_pt_he template:

In [ ]:
! nvflare config -jt ../../../../../job_templates

In [ ]:
! nvflare job create -j /tmp/nvflare/jobs/cifar10_sag_pt_he -w sag_pt_he -sd ../code/fl -force

We can take a look at the client and server configurations and make any changes as desired:

In [ ]:
! cat /tmp/nvflare/jobs/cifar10_sag_pt_he/app/config/config_fed_client.conf

In [ ]:
! cat /tmp/nvflare/jobs/cifar10_sag_pt_he/app/config/config_fed_server.conf

## Prepare Data

Make sure the CIFAR10 dataset is downloaded with the following script:

In [ ]:
! python ../data/download.py --dataset_path /tmp/nvflare/data/cifar10

## Run the Job using POC mode

First ensure that the TenSEAL dependency is installed:

In [ ]:
! pip install nvflare[HE]

To support HE, we need the provisioning process to generate and write the TenSEAL homomorphic encryption contexts for the server and client.
Currently the simulator does not support HE, however we can use the POC command `-he` option to prepare the HE supported POC workspace with the `HEBuilder`:

In [ ]:
! nvflare poc prepare -he

Start the FLARE system in POC mode in another terminal with ```nvflare poc start -ex admin@nvidia.com```

Finally we can submit our HE configured job:

In [ ]:
! nvflare job submit -j /tmp/nvflare/jobs/cifar10_sag_pt_he

Once you are finished with the example, you can shutdown and clean the POC system:

In [ ]:
! nvflare poc stop

In [ ]:
! nvflare poc clean

As an additional resource, see the [CIFAR10 Real World Example](https://github.com/NVIDIA/NVFlare/tree/main/examples/advanced/cifar10/cifar10-real-world) for creating a secure workspace for HE using provisioning instead of POC mode.

Now we will begin to take a look at other workflows besides ScatterAndGather.
First we have the [cse](../cse/cse.ipynb) example, which shows the server-controlled cross-site evaluation workflow.